In [17]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [5]:
keras.__version__

'2.1.6-tf'

In [6]:
# Define a fully-connected network
model = keras.Sequential()

model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [16]:
# Model compile
model.compile(optimizer=tf.train.AdamOptimizer(0.01),
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

In [19]:
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

val_data = np.random.random((100, 32))
val_labels = np.random.random((100, 10)) 


model.fit(data, labels, epochs=10, batch_size=32,
          validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/10
1000/1000 [==============================] - 1s 547us/step - loss: 11.5819 - acc: 0.0880 - val_loss: 11.4309 - val_acc: 0.0700
Epoch 2/10
1000/1000 [==============================] - 0s 235us/step - loss: 11.5768 - acc: 0.1000 - val_loss: 11.4299 - val_acc: 0.1000
Epoch 3/10
1000/1000 [==============================] - 0s 265us/step - loss: 11.5763 - acc: 0.0990 - val_loss: 11.4288 - val_acc: 0.1100
Epoch 4/10
1000/1000 [==============================] - 0s 249us/step - loss: 11.5763 - acc: 0.1010 - val_loss: 11.4308 - val_acc: 0.0700
Epoch 5/10
1000/1000 [==============================] - 0s 269us/step - loss: 11.5765 - acc: 0.1060 - val_loss: 11.4297 - val_acc: 0.0700
Epoch 6/10
1000/1000 [==============================] - 0s 302us/step - loss: 11.5766 - acc: 0.1050 - val_loss: 11.4291 - val_acc: 0.0700
Epoch 7/10
1000/1000 [==============================] - 0s 278us/step - loss: 11.5776 - acc: 0.0870 - val_loss: 11.4288 - val

In [41]:
# Using the Datasets API
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)
dataset = dataset.repeat()

# Need to specify steps_per_epoch
model.fit(dataset, epochs=10, steps_per_epoch=32)

Epoch 1/10
32/32 [==============================] - 1s 16ms/step - loss: 11.5620 - acc: 0.1143
Epoch 2/10
32/32 [==============================] - 0s 7ms/step - loss: 11.5625 - acc: 0.0947
Epoch 3/10
32/32 [==============================] - 0s 9ms/step - loss: 11.5624 - acc: 0.1016
Epoch 4/10
32/32 [==============================] - 0s 10ms/step - loss: 11.5624 - acc: 0.1016
Epoch 5/10
32/32 [==============================] - 0s 9ms/step - loss: 11.5624 - acc: 0.1016
Epoch 6/10
32/32 [==============================] - 0s 9ms/step - loss: 11.5624 - acc: 0.1016
Epoch 7/10
32/32 [==============================] - 0s 10ms/step - loss: 11.5624 - acc: 0.1016
Epoch 8/10
32/32 [==============================] - 0s 9ms/step - loss: 11.5624 - acc: 0.1016
Epoch 9/10
32/32 [==============================] - 0s 9ms/step - loss: 11.5624 - acc: 0.1016
Epoch 10/10
32/32 [==============================] - 0s 9ms/step - loss: 11.5624 - acc: 0.1016


In [35]:
# Evaluation
print(model.metrics_names)
model.evaluate(val_data, val_labels, batch_size=32)

['loss', 'acc']
100/100 [==============================] - 0s 184us/step


[11.429250717163086, 0.12]

In [42]:
# Prediction
y_pred = model.predict(dataset, steps=32)

y_pred.shape

(1000, 10)

In [44]:
# Functional API allows for more complex models

inputs = keras.Input(shape=(32,))  # Returns a placeholder tensor

# A layer instance is callable on a tensor, and returns a tensor.
x = keras.layers.Dense(64, activation='relu')(inputs)
x = keras.layers.Dense(64, activation='relu')(x)
predictions = keras.layers.Dense(10, activation='softmax')(x)

# Instantiate the model given inputs and outputs.
model = keras.Model(inputs=inputs, outputs=predictions)

# The compile step specifies the training configuration.
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Trains for 5 epochs
model.fit(data, labels, batch_size=32, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 1s 877us/step - loss: 11.6597 - acc: 0.1080
Epoch 2/5
1000/1000 [==============================] - 0s 217us/step - loss: 11.6187 - acc: 0.1140
Epoch 3/5
1000/1000 [==============================] - 0s 258us/step - loss: 11.6031 - acc: 0.0920
Epoch 4/5
1000/1000 [==============================] - 0s 270us/step - loss: 11.5925 - acc: 0.0950
Epoch 5/5
1000/1000 [==============================] - 0s 269us/step - loss: 11.5802 - acc: 0.1070


In [46]:
"""
Defining a custom layer

build: Create the weights of the layer. Add weights with the add_weight method.
call: Define the forward pass.
compute_output_shape: Specify how to compute the output shape of the layer given the input shape.
Optionally, a layer can be serialized by implementing the get_config method and the from_config class method.


"""


class MyLayer(keras.layers.Layer):

  def __init__(self, output_dim, **kwargs):
    self.output_dim = output_dim
    super(MyLayer, self).__init__(**kwargs)

  def build(self, input_shape):
    shape = tf.TensorShape((input_shape[1], self.output_dim))
    # Create a trainable weight variable for this layer.
    self.kernel = self.add_weight(name='kernel',
                                  shape=shape,
                                  initializer='uniform',
                                  trainable=True)
    # Be sure to call this at the end
    super(MyLayer, self).build(input_shape)

  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)

  def compute_output_shape(self, input_shape):
    shape = tf.TensorShape(input_shape).as_list()
    shape[-1] = self.output_dim
    return tf.TensorShape(shape)

  def get_config(self):
    base_config = super(MyLayer, self).get_config()
    base_config['output_dim'] = self.output_dim
    return base_config

  @classmethod
  def from_config(cls, config):
    return cls(**config)


# Create a model using the custom layer
model = keras.Sequential([MyLayer(10),
                          keras.layers.Activation('softmax')])

# The compile step specifies the training configuration
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Trains for 5 epochs.
model.fit(data, labels, batch_size=32, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 1s 791us/step - loss: 11.6084 - acc: 0.0970
Epoch 2/5
1000/1000 [==============================] - 0s 194us/step - loss: 11.5872 - acc: 0.0870
Epoch 3/5
1000/1000 [==============================] - 0s 231us/step - loss: 11.5807 - acc: 0.1070
Epoch 4/5
1000/1000 [==============================] - 0s 238us/step - loss: 11.5791 - acc: 0.1240
Epoch 5/5
1000/1000 [==============================] - 0s 240us/step - loss: 11.5773 - acc: 0.1170


In [15]:
# Create a sigmoid layer
keras.layers.Dense(64, activation='sigmoid')
# Or
keras.layers.Dense(64, activation=tf.sigmoid)

# Linear layer with L1 reg. applied to kernel matrix
keras.layers.Dense(64, kernel_regularizer=keras.regularizers.l1(0.01))
# Linear layer with L2 reg. applied to bias
keras.layers.Dense(64, bias_regularizer=keras.regularizers.l2(0.01))


# Linear layer with kernel initialized to random orthogonal matrix
keras.layers.Dense(64, kernel_initializer="orthogonal")

# Linear layer with kernel initialized to 2.0s
keras.layers.Dense(64, kernel_initializer=keras.initializers.constant(2.0))